#### Varianta 1: sentiment pe textele in engleza

In [6]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sentiment = SentimentIntensityAnalyzer()

text_1 = "By rejecting God, you are rejecting goodness, kindness, and love itself."
text_2 = "We, the people, are going to work together to achieve justice, unity, and integrity."
text_3 = "Ms. Baker assigned me a lot of homework because she’s a witch!"

sent_1 = sentiment.polarity_scores(text_1)
sent_2 = sentiment.polarity_scores(text_2)
sent_3 = sentiment.polarity_scores(text_3)

print("Sentiment of text 1:", sent_1)
print("Sentiment of text 2:", sent_2)
print("Sentiment of text 3:", sent_3)

Sentiment of text 1: {'neg': 0.258, 'neu': 0.215, 'pos': 0.528, 'compound': 0.743}
Sentiment of text 2: {'neg': 0.0, 'neu': 0.667, 'pos': 0.333, 'compound': 0.7184}
Sentiment of text 3: {'neg': 0.202, 'neu': 0.798, 'pos': 0.0, 'compound': -0.4199}


#### Varianta 2: sentiment pe textele in romana, dar..

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "dumitrescustefan/bert-base-romanian-cased-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    sentiment = torch.argmax(logits).item()
    labels = {0: "Negative", 1: "Neutral", 2: "Positive"}
    return labels[sentiment]

text = "Îmi place foarte mult acest film!"
print(analyze_sentiment(text))

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

C:\Users\mraic\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mraic\.cache\huggingface\hub\models--dumitrescustefan--bert-base-romanian-cased-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/397k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/500M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dumitrescustefan/bert-base-romanian-cased-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Negative


#### Varianta 3: sentiment pe textele in romana, dar..


In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "dumitrescustefan/bert-base-romanian-uncased-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    sentiment = torch.argmax(logits).item()
    labels = {0: "Negative", 1: "Neutral", 2: "Positive"}
    return labels[sentiment]

text = "Îmi place foarte mult acest film!"
print(analyze_sentiment(text))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dumitrescustefan/bert-base-romanian-uncased-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Negative


#### Varianta 4: sentiment pe textele in romana, ro_sentiment, dar..
 
https://huggingface.co/readerbench/ro-sentiment
Output labels:

LABEL_0 = Negative Sentiment
LABEL_1 = Positive Sentiment

In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "readerbench/ro-sentiment"

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def sentiment(text):
    """Assess the sentiment of the input text."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits.squeeze().cpu()
    return logits.tolist()

# Example usage
score = sentiment("Îmi place foarte mult acest film!")
print(f"Sentiment score: {score}")

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

C:\Users\mraic\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mraic\.cache\huggingface\hub\models--readerbench--ro-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/245k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/842k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/781 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/460M [00:00<?, ?B/s]

Sentiment score: [0.24193798005580902, -1.1060422658920288]


### Varianta finala

In [1]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

df = pd.read_csv("../data/all/combined_lfud_huggingface_binary.csv")

analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):
    if pd.isna(text):
        print("There is a nan value")
    score = analyzer.polarity_scores(text)['compound']
    if score >= 0.05:
        return "positive"
    elif score <= -0.05:
        return "negative"
    else:
        return "neutral"

# Apply sentiment analysis to 'source_article'
df['sentiment'] = df['source_article'].apply(get_sentiment)

In [3]:
df.to_csv("../data/all/combined_lfud_huggingface_binary_sent.csv", index=False)
print(df.head())

                                      source_article logical_fallacies  \
0  Such misrepresentations are now commonplace in...           fallacy   
1  If we use just one more can of hairspray this ...           fallacy   
2  That doesn ’ t mean pollution controls are fut...           fallacy   
3                         The Earth is not warming .           fallacy   
4     McDonald's Hamburgers: over 99 billion served.           fallacy   

                                   source_article_ro proposition  \
0  Astfel de denaturari sunt acum obisnuite in an...         NaN   
1  Dacă mai folosim o singură cutie de fixativ lu...         NaN   
2  Asta nu înseamnă că controalele poluării sunt ...         NaN   
3                         Pământul nu se încălzește.         NaN   
4  Hamburgeri McDonald's: peste 99 de miliarde se...         NaN   

  proposition_ro sentiment  
0            NaN  negative  
1            NaN  negative  
2            NaN   neutral  
3            NaN  negative  
4

In [4]:
import pandas as pd
data = pd.read_csv("../data/all/combined_lfud_huggingface_binary_sent.csv")

In [5]:
data.value_counts(data["sentiment"])

sentiment
negative    1978
positive    1938
neutral     1790
Name: count, dtype: int64